In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np

from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

In [25]:
#Data augumentattion, normaliztion(데이토 증가, 일반화) : 과적합 방지 위해 사용
data_transforms = {
   'train' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]),
    'test' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        }

#데이터를 불러오는 경로를 설정
batch_size = 4

data_dir='./animal'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x])
                 for x in ['train', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4,
                                             shuffle = True, num_workers=0)
                for x in ['train', 'test']}

dataset_sizes = {x:len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes


data_dir1='./animal/train'
trainloader = torch.utils.data.DataLoader(data_dir1, batch_size = batch_size,
                                         shuffle=True, num_workers=2)


data_dir2='./animal/test'
testloader = torch.utils.data.DataLoader(data_dir2, batch_size = batch_size,
                                         shuffle=True, num_workers=2)
classes = ('cats', 'dogs')

#CUDA(GPU 가속화) 사용이 가능하게 만드는 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [26]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [27]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9)

In [28]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch +1}, {i+1:5d} loss : {running_loss / 2000:.3f}]')
            running_loss = 0.0
            
print("Traning finish")

ValueError: too many values to unpack (expected 2)